<a href="https://colab.research.google.com/github/plthiyagu/Personnel/blob/master/cuDF_RAPIDS_GPU_Accelerated_Dataframe_Library.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget "http://eforexcel.com/wp/wp-content/uploads/2020/09/5m-Sales-Records.zip"

--2020-09-13 23:31:09--  http://eforexcel.com/wp/wp-content/uploads/2020/09/5m-Sales-Records.zip
Resolving eforexcel.com (eforexcel.com)... 162.214.80.24
Connecting to eforexcel.com (eforexcel.com)|162.214.80.24|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 201708941 (192M) [application/zip]
Saving to: ‘5m-Sales-Records.zip’

5m-Sales-Records.zi 100%[===================>] 192.36M  12.7MB/s    in 7.9s    

2020-09-13 23:31:17 (24.4 MB/s) - ‘5m-Sales-Records.zip’ saved [201708941/201708941]



In [2]:
import zipfile
import os
with zipfile.ZipFile("/content/5m-Sales-Records.zip", 'r') as zip_ref:
    zip_ref.extractall()


In [3]:
print ("Size of file is {} MB".format(os.path.getsize("/content/5m Sales Records.csv") >> 20))

Size of file is 595 MB


In [4]:
!nvidia-smi

Sun Sep 13 23:31:29 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.66       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!bash rapidsai-csp-utils/colab/rapids-colab.sh stable

import sys, os

dist_package_index = sys.path.index('/usr/local/lib/python3.6/dist-packages')
sys.path = sys.path[:dist_package_index] + ['/usr/local/lib/python3.6/site-packages'] + sys.path[dist_package_index:]
sys.path
exec(open('rapidsai-csp-utils/colab/update_modules.py').read(), globals())

Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 185 (delta 5), reused 0 (delta 0), pack-reused 171
Receiving objects: 100% (185/185), 57.14 KiB | 1.39 MiB/s, done.
Resolving deltas: 100% (67/67), done.
PLEASE READ
********************************************************************************************************
Changes:
1. IMPORTANT CHANGES: RAPIDS on Colab will be pegged to 0.14 Stable until further notice.
2. Default stable version is now 0.14.  Nightly will redirect to 0.14.
3. You can now declare your RAPIDSAI version as a CLI option and skip the user prompts (ex: '0.14' or '0.15', between 0.13 to 0.14, without the quotes): 
        "!bash rapidsai-csp-utils/colab/rapids-colab.sh <version/label>"
        Examples: '!bash rapidsai-csp-utils/colab/rapids-colab.sh 0.14', or '!bash rapidsai-csp-utils/colab/rapids-colab.sh stable', or '!bash rapi

In [6]:
import time
import numpy as np
import pandas as pd
import cudf

In [7]:
s = time.time()
df_pandas = pd.read_csv("/content/5m Sales Records.csv")
e = time.time()
pd_time = e - s
print("Pandas Loading Time = {}".format(pd_time))

Pandas Loading Time = 18.66586136817932


In [8]:
df_pandas.head()

,Region,Country,Item Type,Sales Channel,Order Priority,Order Date,Order ID,Ship Date,Units Sold,Unit Price,Unit Cost,Total Revenue,Total Cost,Total Profit
0,Australia and Oceania,Palau,Office Supplies,Online,H,3/6/2016,517073523,3/26/2016,2401,651.21,524.96,1563555.21,1260428.96,303126.25
1,Europe,Poland,Beverages,Online,L,4/18/2010,380507028,5/26/2010,9340,47.45,31.79,443183.00,296918.60,146264.40
2,North America,Canada,Cereal,Online,M,1/8/2015,504055583,1/31/2015,103,205.70,117.11,21187.10,12062.33,9124.77
3,Europe,Belarus,Snacks,Online,C,1/19/2014,954955518,2/27/2014,1414,152.58,97.44,215748.12,137780.16,77967.96
4,Middle East and North Africa,Oman,Cereal,Offline,H,4/26/2019,970755660,6/2/2019,7027,205.70,117.11,1445453.90,822931.97,622521.93


In [9]:
s = time.time()
df_cudf = cudf.read_csv("/content/5m Sales Records.csv")
e = time.time()
cuDF_time = e - s 
print("cuDF Loading Time = {}".format(cuDF_time))

cuDF Loading Time = 1.5041406154632568


In [10]:
pd_time > cuDF_time

True

In [11]:
pd_time / cuDF_time

12.409651847896193

In [12]:
s = time.time()
df_pandas_concat = pd.concat([df_pandas for _ in range(5)])
e = time.time()
pd_time = e - s
print("Pandas Concat Time = {}".format(pd_time))

Pandas Concat Time = 4.480583429336548


In [13]:
s = time.time()
df_cudf_concat = cudf.concat([df_cudf for _ in range(5)])
e = time.time()
cuDF_time = e - s
print("cuDF Concat Time = {}".format(cuDF_time))


cuDF Concat Time = 0.7276957035064697


In [14]:
pd_time > cuDF_time

True

In [15]:
pd_time / cuDF_time

6.157221222753464

In [16]:
s = time.time()
print(df_pandas.groupby("Region")["Total Profit"].sum())
e = time.time()
pd_time = e - s
print("Pandas GroupBy Calculation Time = {}".format(pd_time))

Region
Asia                                 2.866868e+11
Australia and Oceania                1.592897e+11
Central America and the Caribbean    2.123582e+11
Europe                               5.089727e+11
Middle East and North Africa         2.440779e+11
North America                        4.249197e+10
Sub-Saharan Africa                   5.095223e+11
Name: Total Profit, dtype: float64
Pandas GroupBy Calculation Time = 0.3349440097808838


In [17]:
s = time.time()
print(df_cudf.groupby("Region")["Total Profit"].sum())
e = time.time()
cuDF_time  = e - s
print("Pandas GroupBy Calculation Time = {}".format(cuDF_time))

Region
Asia                                 2.866868e+11
Australia and Oceania                1.592897e+11
Central America and the Caribbean    2.123582e+11
Europe                               5.089727e+11
Middle East and North Africa         2.440779e+11
North America                        4.249197e+10
Sub-Saharan Africa                   5.095223e+11
Name: Total Profit, dtype: float64
Pandas GroupBy Calculation Time = 0.09269452095031738


In [18]:
pd_time > cuDF_time

True

In [19]:
pd_time / cuDF_time

3.613417560682015